## Convert L3 NetCDFs to GeoTIFF

This notebook is set up so you can either convert files one by one with a filepath, in a batch as long as they're in the same directory, or from a single/batch of earthaccess paths. Simply change the necessary paramaters in the ```User Input``` cell and run the notebook! Make sure to set the ```single_path``` parameter to ```None``` if you're batch processing, otherwise the code will convert the single file path end there. 

In [1]:
from pathlib import Path

import earthaccess
import glob
import rasterio
import rioxarray as rio
import xarray as xr
from os.path import basename

def nc_to_cog(filepath, output_dir=None):
    """
    Takes in a filepath to a PACE L3 SFREFL or LANDVI netcdf file and converts 
        it to a COG. Will place output file in the same directory as the input 
        file as default
    Args:
        filepath: Path to file for conversion 
        output_dir: Optional, path to put converted files.
    Returns:
        COG file with same name as input file but .tif suffix
    """
    if "SFREFL" in str(filepath):
        ds = xr.open_dataset(filepath).rhos.transpose("wavelength", ...)
    elif "LANDVI" in str(filepath):
        ds = xr.open_dataset(filepath).drop_vars("palette")

    ds = ds.rio.write_crs("epsg:4326")
    if output_dir is not None:
        tif_path = Path(filepath).with_suffix(".tif")
        outpath = f"{output_dir}/{basename(tif_path)}"
    else:
        outpath = Path(filepath).with_suffix(".tif")
    ds.rio.to_raster(outpath, driver="COG")
    print(f"File saved to {outpath}")

In [4]:
#################################################
#                 User Input                    #
#################################################

# Are you batch processing? 
batch = False                                            # If not, False

# Do you have the file directory already?
file_dir = None                                          # If not, None

# If not batch processing, provide the path for the file you want to convert
single_path = None                                      # If not, None

# Do you need to download the data through earthaccess (for batch or not)?
need_data = False                                        # If not, False

# If you need to download data, provide search params and an output file directory
download_dir = "data"
short_names = ["PACE_OCI_L3M_LANDVI",]                  # "PACE_OCI_L3M_SFREFL", or either with _NRT for more recent data
tspan = ("2024-03-05", "2026-12-31")                    # e.g., ("2024-12-07", "2024-12-18")
granule = "*.20240601_20240630.*.0p1deg.*"              # e.g., "*.20240601_20240630.*.0p1deg.*"

# Optionally, provide an output directory for the converted files. Defaults to input file path
output_dir = "data"

In [5]:
#################################################
#       Should not need to touch this code      #
#################################################
if need_data:
    batch=False

if not batch and not need_data:
    if single_path is not None:
        nc_to_cog(single_path, output_dir=output_dir)
    else:
        print("Please provide the path for the file you want to convert and rerun the cells")
elif batch:
    if file_dir is None:
        print("Please provide the directory for the files you want to convert in the cell above and rerun these cells")
    else:
        paths = sorted(glob.glob(f"{file_dir}/PACE*.nc"))
        for path in paths:
            nc_to_cog(path, output_dir=output_dir)
elif need_data:
    results = earthaccess.search_data(
            short_name=short_names,
            granule_name=granule,
            temporal=tspan,
        )
    print(f"Downloading the following files to {download_dir}:")
    for item in results:
        display(item)
    l3_paths = earthaccess.download(results, local_path=download_dir)
    for path in l3_paths:
        nc_to_cog(path, output_dir=output_dir)

Please provide the path for the file you want to convert and rerun the cells
